In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ConvNeXtSmall

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ["acne", "pigmentation", "wrinkles"]
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/"

df = pd.read_csv(os.path.join(DATA_ROOT, "labels.csv"))
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

In [3]:
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

pos_counts = train_df[CLASSES].sum().values

In [4]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
], name="data_augmentation")

def parse_function(filename, labels):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image_resized = tf.image.resize(image, IMG_SIZE)
    return image_resized, labels

def create_dataset(df, batch_size, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["filename"].values, df[CLASSES].values.astype(np.float32))
    )
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.cache()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_ds = create_dataset(train_df, BATCH_SIZE, augment=True)
val_ds = create_dataset(val_df, BATCH_SIZE, augment=False)
test_ds = create_dataset(test_df, BATCH_SIZE, augment=False)

print("tf.data pipelines created successfully with augmentation enabled for training.")

tf.data pipelines created successfully with augmentation enabled for training.


In [8]:
def build_convnext(input_shape, num_classes):
    base_model = ConvNeXtSmall(
        include_top=False,
        input_shape=input_shape,
        weights="imagenet"
    )


    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", dtype='float32')(x)
    model = Model(inputs, outputs, name="convnext_model")
    return model

tf.keras.backend.clear_session()
convnext_model = build_convnext(input_shape=IMG_SIZE + (3,), num_classes=len(CLASSES))

In [9]:
def weighted_bce(y_true, y_pred, smooth=0.05):
    y_true = y_true * (1.0 - smooth) + 0.5 * smooth
    bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
    pos = tf.constant(pos_counts, dtype=tf.float32)
    neg = len(train_df) - pos
    w_pos = neg / tf.maximum(pos, 1.0)
    w_neg = tf.ones_like(pos)
    weights = y_true * w_pos + (1.0 - y_true) * w_neg
    return tf.reduce_mean(bce * weights)

# Callbacks
CONVNEXT_MODEL_PATH = os.path.join(DATA_ROOT, "convnext_skin_model.keras")
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=7, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.2, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(CONVNEXT_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True)
]

#STAGE 1: FEATURE EXTRACTION
print("\nStage 1: Training the classification head...")

#Freeze the base model
convnext_model.get_layer("convnext_small").trainable = False

#Compiling the model with a regular learning rate
convnext_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4),
    loss=weighted_bce,
    metrics=["acc", "auc"]
)

#Training only the head for a few epochs
history_head = convnext_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)

#STAGE 2: FINE-TUNING
print("\nStage 2: Unfreezing and fine-tuning the entire model...")

#Unfreeze the base model
convnext_model.get_layer("convnext_small").trainable = True

#Re-compile the model with a VERY LOW learning rate
convnext_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-4),
    loss=weighted_bce,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

history_fine_tune = convnext_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks,
    initial_epoch=len(history_head.history['loss']),
    verbose=1
)


Stage 1: Training the classification head...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 73s 440ms/step - acc: 0.3715 - auc: 0.5118 - loss: 1.4169 - val_acc: 0.4350 - val_auc: 0.8104 - val_loss: 1.0962
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 29s 255ms/step - acc: 0.3893 - auc: 0.6419 - loss: 1.1319 - val_acc: 0.4644 - val_auc: 0.8230 - val_loss: 1.0421
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 29s 251ms/step - acc: 0.4197 - auc: 0.7303 - loss: 1.0713 - val_acc: 0.4985 - val_auc: 0.8334 - val_loss: 1.0029
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 28s 247ms/step - acc: 0.4448 - auc: 0.7594 - loss: 1.0433 - val_acc: 0.4783 - val_auc: 0.8430 - val_loss: 0.9754
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 29s 250ms/step - acc: 0.4282 - auc: 0.7689 - loss: 1.0260 - val_acc: 0.4923 - val_auc: 0.8467 - val_loss: 0.9569
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 29s 249ms/step - acc: 0.4697 - auc: 0.7966 - loss: 0.9972 - val_acc: 0.4892 - val_auc: 0.8540 - val_loss: 0.9388
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━

In [10]:
print(f"Loading best model from: {CONVNEXT_MODEL_PATH}")
loaded_model = tf.keras.models.load_model(
    CONVNEXT_MODEL_PATH,
    custom_objects={"weighted_bce": weighted_bce}
)
print("Model loaded successfully!")

print("\nEvaluating the final model on the test set...")
test_results = loaded_model.evaluate(test_ds)
print("\nFinal Test Set Evaluation Results")
for metric, value in zip(loaded_model.metrics_names, test_results):
    print(f"{metric}: {value:.4f}")

Loading best model from: /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/convnext_skin_model.keras
Model loaded successfully!

Evaluating the final model on the test set...
24/24 ━━━━━━━━━━━━━━━━━━━━ 317s 13s/step - acc: 0.9498 - auc: 0.9823 - loss: 0.4058 - precision: 0.8362 - recall: 0.9128

Final Test Set Evaluation Results
loss: 0.3960
compile_metrics: 0.9561
